In [ ]:
!pip install evaluate sacrebleu
import json
import re
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from collections import Counter
from tqdm import tqdm
import sacrebleu

from transformers import (
    AutoTokenizer,
    T5ForConditionalGeneration,
    T5Config,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)

# ==========================================
# 0. 환경 설정 및 시드 고정
# ==========================================
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> 환경 설정 완료. Device: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 10.5 MB/s eta 0:00:00
>>> 환경 설정 완료. Device: cuda


In [ ]:
# ==========================================
# 1. 평가 지표 함수 (PARENT & BLEU)
# ==========================================
def get_ngrams(segment, max_order):
    ngram_counts = Counter()
    for order in range(1, max_order + 1):
        for i in range(0, len(segment) - order + 1):
            ngram = tuple(segment[i:i+order])
            ngram_counts[ngram] += 1
    return ngram_counts

def parent_score(predictions, references, tables):
    """
    ToTTo 리더보드용 PARENT 스코어 계산
    """
    total_precision, total_recall, total_f1 = 0.0, 0.0, 0.0
    max_order = 4
    smoothing = 1e-13

    for pred, refs, table in zip(predictions, references, tables):
        pred_tokens = pred.strip().split()
        ref_tokens_list = [r.strip().split() for r in refs]
        table_tokens = [str(t) for t in table]

        pred_ngrams = get_ngrams(pred_tokens, max_order)
        ref_ngrams_list = [get_ngrams(ref, max_order) for ref in ref_tokens_list]
        table_ngrams = get_ngrams(table_tokens, max_order)

        # Precision
        numerator_prec = 0.0
        denominator_prec = sum(pred_ngrams.values()) + smoothing
        for ngram, count in pred_ngrams.items():
            prob_in_table = 1.0 if ngram in table_ngrams else 0.0
            prob_in_ref = 0.0
            for ref_ngrams in ref_ngrams_list:
                prob_in_ref = max(prob_in_ref, min(1.0, ref_ngrams.get(ngram, 0) / count))
            w_prob = prob_in_table + prob_in_ref * (1.0 - prob_in_table)
            numerator_prec += count * w_prob
        precision = numerator_prec / denominator_prec

        # Recall
        best_recall = 0.0
        for ref_ngrams in ref_ngrams_list:
            numerator_rec = 0.0
            denominator_rec = sum(ref_ngrams.values()) + smoothing
            for ngram, count in ref_ngrams.items():
                if ngram in table_ngrams:
                     if ngram in pred_ngrams:
                        numerator_rec += min(count, pred_ngrams[ngram])
            best_recall = max(best_recall, numerator_rec / denominator_rec)
        recall = best_recall

        f1 = 0.0
        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)

        total_precision += precision
        total_recall += recall
        total_f1 += f1

    n = len(predictions)
    return total_precision/n, total_recall/n, total_f1/n

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple): preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # 학습 중에는 빠르고 간단한 BLEU만 확인
    result = sacrebleu.corpus_bleu(decoded_preds, decoded_labels)
    return {"bleu": result.score}

In [ ]:
# ==========================================
# 2. 모델 정의: SafeStructureT5 (Concat + Identity Init)
# ==========================================
class SafeStructureT5(T5ForConditionalGeneration):
    def __init__(self, config, header_init_matrix=None):
        super().__init__(config)

        # 1. 헤더 임베딩 (padding_idx=0으로 설정하여 None값 처리)
        if header_init_matrix is not None:
            self.row_header_embeddings = nn.Embedding.from_pretrained(header_init_matrix, freeze=False, padding_idx=0)
            self.col_header_embeddings = nn.Embedding.from_pretrained(header_init_matrix, freeze=False, padding_idx=0)
        else:
            self.row_header_embeddings = nn.Embedding(2048, config.d_model, padding_idx=0)
            self.col_header_embeddings = nn.Embedding(2048, config.d_model, padding_idx=0)

        # 2. 차원 축소 (Concat: Word + Row + Col -> Original Dim)
        self.feature_projection = nn.Linear(config.d_model * 3, config.d_model, bias=False)
        self.norm = nn.LayerNorm(config.d_model) # 학습 안정성을 위한 정규화

        # 3. [핵심] Identity Initialization (학습 초기 Highlighted Cell 보호)
        self._init_projection_weights()

    def _init_projection_weights(self):
        hidden_dim = self.config.d_model
        with torch.no_grad():
            # 단어(Word) 정보는 100% 통과 (Identity Matrix)
            self.feature_projection.weight[:, :hidden_dim] = torch.eye(hidden_dim)
            # 구조(Structure) 정보는 0% 통과 (차단) - 학습하며 점진적 반영
            self.feature_projection.weight[:, hidden_dim:] = 0.0

    def forward(self, input_ids=None, attention_mask=None, labels=None, row_ids=None, col_ids=None, **kwargs):
        # [Safe Guard 1] Transformers 최신 버전 호환성 (에러 방지)
        kwargs.pop("num_items_in_batch", None)

        # [Safe Guard 2] Generation 등에서 input_ids가 없을 경우
        if input_ids is None:
             return super().forward(input_ids=None, attention_mask=attention_mask, labels=labels, **kwargs)

        word_embeddings = self.shared(input_ids)

        # [Safe Guard 3] row_ids/col_ids가 없을 경우 0 벡터 처리
        if row_ids is not None and col_ids is not None:
            row_embeddings = self.row_header_embeddings(row_ids)
            col_embeddings = self.col_header_embeddings(col_ids)
        else:
            row_embeddings = torch.zeros_like(word_embeddings)
            col_embeddings = torch.zeros_like(word_embeddings)

        # [핵심 로직] Concat (옆으로 붙이기)
        fused_embeddings = torch.cat([word_embeddings, row_embeddings, col_embeddings], dim=-1)

        # Projection & Norm
        fused_embeddings = self.feature_projection(fused_embeddings)
        fused_embeddings = self.norm(fused_embeddings)

        return super().forward(
            inputs_embeds=fused_embeddings, # input_ids 대신 임베딩 전달
            attention_mask=attention_mask,
            labels=labels,
            **kwargs
        )

In [ ]:
# ==========================================
# 3. 데이터 처리 유틸리티
# ==========================================
def parse_totto_string(input_str):
    # 정규식으로 메타데이터와 셀 정보 파싱
    page_match = re.search(r'\[PAGE\](.*?)(?=\[SEC\]|\[TEXT\]|\[CELL\]|$)', input_str)
    sec_match = re.search(r'\[SEC\](.*?)(?=\[TEXT\]|\[CELL\]|$)', input_str)

    page_title = page_match.group(1).strip() if page_match else ""
    section_title = sec_match.group(1).strip() if sec_match else ""

    # Cell 파싱: [CELL] 값 [TYPE] ... [R_HEAD] 값 [C_HEAD] 값
    cell_pattern = re.compile(r'\[CELL\](.*?)\[TYPE\].*?\[R_HEAD\](.*?)\[C_HEAD\](.*?)(?=\[CELL\]|$)')
    cells = []
    for match in cell_pattern.finditer(input_str):
        cells.append({
            'val': match.group(1).strip(),
            'row': match.group(2).strip(),
            'col': match.group(3).strip()
        })
    return {'page': page_title, 'section': section_title, 'cells': cells}

def prepare_semantic_headers(df, base_model_name="t5-base"):
    print(">>> 헤더 정보 스캔 및 임베딩 매트릭스 생성 중...")
    unique_headers = set(["None"])

    # 샘플 데이터에서 헤더 수집
    for text in tqdm(df['input']):
        parsed = parse_totto_string(text)
        for c in parsed['cells']:
            unique_headers.add(c['row'])
            unique_headers.add(c['col'])

    sorted_headers = ["None"] + sorted([h for h in unique_headers if h != "None"])
    header2id = {h: i for i, h in enumerate(sorted_headers)}

    # Base T5 모델을 이용해 헤더 텍스트를 벡터로 변환 (Semantic Initialization)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    temp_model = T5ForConditionalGeneration.from_pretrained(base_model_name)
    wte = temp_model.get_input_embeddings()

    init_matrix = torch.zeros((len(sorted_headers), temp_model.config.d_model))

    print(">>> 의미론적 헤더 임베딩 초기화 중...")
    for i, h in enumerate(tqdm(sorted_headers)):
        if i == 0: continue # None은 0 벡터 유지
        ids = tokenizer.encode(h, add_special_tokens=False)
        if not ids: continue
        with torch.no_grad():
            vecs = wte(torch.tensor([ids]))
            init_matrix[i] = torch.mean(vecs, dim=1).squeeze(0)

    return header2id, init_matrix

class StructureT5Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, header2id, max_len=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.header2id = header2id
        self.max_len = max_len
        self.hl_start = tokenizer.convert_tokens_to_ids("<hl>")
        self.hl_end = tokenizer.convert_tokens_to_ids("</hl>")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        parsed = parse_totto_string(item['input'])

        # 입력 텍스트 구성: Context + <hl> Highlighted Cells </hl>
        context = f"Page: {parsed['page']} | Section: {parsed['section']} "
        cell_texts = []
        map_list = []

        for c in parsed['cells']:
            cell_texts.append(f"<hl> {c['val']} </hl>")
            # 헤더 텍스트를 ID로 변환 (없으면 0)
            map_list.append((self.header2id.get(c['row'], 0), self.header2id.get(c['col'], 0)))

        full_text = context + " ".join(cell_texts)

        # 토큰화
        tokenized = self.tokenizer(full_text, max_length=self.max_len, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = tokenized['input_ids'].squeeze(0)

        # 구조 정보(Row/Col ID) 매핑
        row_ids = torch.zeros_like(input_ids)
        col_ids = torch.zeros_like(input_ids)

        cell_idx = 0
        in_hl = False
        for i, tid in enumerate(input_ids):
            if tid == self.hl_start:
                in_hl = True; continue
            if tid == self.hl_end:
                in_hl = False; cell_idx += 1; continue

            # <hl> 태그 내부에 있는 토큰에만 헤더 정보 부여
            if in_hl and cell_idx < len(map_list):
                row_ids[i] = map_list[cell_idx][0]
                col_ids[i] = map_list[cell_idx][1]

        # 라벨 처리
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(item['target'], max_length=128, padding="max_length", truncation=True, return_tensors="pt")['input_ids'].squeeze(0)
            labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": tokenized['attention_mask'].squeeze(0),
            "row_ids": row_ids,
            "col_ids": col_ids,
            "labels": labels
        }

In [ ]:
# ==========================================
# 4. 실행 파이프라인 (Main)
# ==========================================

# [경로 설정]
TRAIN_PATH = '/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/totto_data/totto_preprocessed_train.json'
VAL_PATH = '/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/totto_data/totto_preprocessed_dev.json'
OUTPUT_DIR = '/content/drive/MyDrive/멋사 NLP 자연어처리/팀프로젝트_2/safe_structure_t5_final'

print(">>> 1. 데이터 로드 및 샘플링...")
train_df = pd.read_json(TRAIN_PATH)
val_df = pd.read_json(VAL_PATH)

# [요청사항 반영] 학습 120,000개, 검증 7,700개 샘플링
train_df = train_df.sample(n=min(120000, len(train_df)), random_state=42)
val_df = val_df.sample(n=min(7700, len(val_df)), random_state=42)
print(f"    Train Samples: {len(train_df)}")
print(f"    Val Samples:   {len(val_df)}")

# 헤더 초기화 매트릭스 생성
header2id, init_matrix = prepare_semantic_headers(train_df)

# 토크나이저 및 모델 설정
print(">>> 2. 모델 및 토크나이저 초기화...")
tokenizer = AutoTokenizer.from_pretrained("t5-base")
tokenizer.add_special_tokens({'additional_special_tokens': ['<hl>', '</hl>']})

config = T5Config.from_pretrained("t5-base")
model = SafeStructureT5.from_pretrained("t5-base", config=config, header_init_matrix=init_matrix)
model.resize_token_embeddings(len(tokenizer)) # 특수 토큰 추가 반영

# 데이터셋 생성
train_dataset = StructureT5Dataset(train_df, tokenizer, header2id)
val_dataset = StructureT5Dataset(val_df, tokenizer, header2id)

# 학습 인자 설정
print(">>> 3. Trainer 설정...")
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    num_train_epochs=3,
    logging_steps=500,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    remove_unused_columns=False,        # [중요] row_ids, col_ids 전달을 위해 필수
    bf16=True,                          # A100 사용 시 True 권장
    report_to="none"
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=-100),
    compute_metrics=compute_metrics
)

# 학습 시작
print(">>> 4. 학습 시작!")
trainer.train()

# 모델 저장
print(f">>> 5. 모델 저장: {OUTPUT_DIR}")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

>>> 1. 데이터 로드 및 샘플링...
    Train Samples: 120000
    Val Samples:   7700
>>> 헤더 정보 스캔 및 임베딩 매트릭스 생성 중...


100%|██████████| 120000/120000 [00:01<00:00, 79244.93it/s]


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,Bleu
500,15.564200,3.513478,11.478744
1000,14.738800,3.431510,11.478744
1500,14.469300,3.384413,9.287529
2000,14.271800,3.351873,11.478744
2500,14.118400,3.324115,11.478744
3000,14.000900,3.302693,11.478744
3500,13.919000,3.286271,0.000000
4000,13.761000,3.272782,9.287529
4500,13.625900,3.261792,9.287529
5000,13.566300,3.249663,9.287529


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

Step,Training Loss,Validation Loss,Bleu
500,15.564200,3.513478,11.478744
1000,14.738800,3.431510,11.478744
1500,14.469300,3.384413,9.287529
2000,14.271800,3.351873,11.478744
2500,14.118400,3.324115,11.478744
3000,14.000900,3.302693,11.478744
3500,13.919000,3.286271,0.000000
4000,13.761000,3.272782,9.287529
4500,13.625900,3.261792,9.287529
5000,13.566300,3.249663,9.287529


/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

KeyboardInterrupt: 

In [ ]:
# ==========================================
# 5. 최종 평가 (PARENT Score & Visualization)
# ==========================================
print("\n>>> 6. 최종 평가 (PARENT Score 계산)...")

# 예측 생성
preds_output = trainer.predict(val_dataset)
decoded_preds = tokenizer.batch_decode(preds_output.predictions, skip_special_tokens=True)
decoded_preds = [p.strip() for p in decoded_preds]

# PARENT 계산을 위한 참조 데이터 준비
references = []
tables = []
for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):
    references.append([row['target'].strip()])
    parsed = parse_totto_string(row['input'])
    table_tokens = []
    table_tokens.extend(parsed['page'].split())
    table_tokens.extend(parsed['section'].split())
    for cell in parsed['cells']:
        table_tokens.extend(cell['val'].split())
    tables.append(table_tokens)

precision, recall, f1 = parent_score(decoded_preds, references, tables)

print("\n" + "="*40)
print(f"📊 FINAL TEST RESULTS (Sampled Val)")
print(f"   PARENT Precision: {precision:.4f}")
print(f"   PARENT Recall:    {recall:.4f}")
print(f"   PARENT F1:        {f1:.4f}")
print("="*40)

# 결과 샘플 확인
print("\n>>> 🎲 생성 결과 샘플 확인:")
indices = random.sample(range(len(decoded_preds)), 3)
for i, idx in enumerate(indices):
    print(f"\n[Sample {i+1}]")
    print(f"▶ Reference:  {references[idx][0]}")
    print(f"▶ Prediction: {decoded_preds[idx]}")
    print("-" * 50)

# 결과 파일 저장
with open(f"{OUTPUT_DIR}/final_results.json", "w") as f:
    json.dump({
        "parent_metrics": {"precision": precision, "recall": recall, "f1": f1},
        "samples": [{"ref": references[i][0], "pred": decoded_preds[i]} for i in indices]
    }, f, indent=2)

print(">>> 모든 작업 완료! 수고하셨습니다. 👍")

In [ ]:
# 결과 샘플 확인
print("\n>>> 🎲 생성 결과 샘플 확인:")
indices = random.sample(range(len(decoded_preds)), 3)  # <--- 여기 숫자 '3'